In [ ]:
import tensorflow 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import os
import csv
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
root_dir='../input/taiwanese-food-101/tw_food_101/tw_food_101'
train_dir = root_dir + '/train'
test_dir = root_dir + '/test'

In [ ]:
%matplotlib inline
rows = 17
cols = 6
fig, ax = plt.subplots(rows, cols, frameon=False, figsize=(15, 25))
food_dirs = os.listdir(train_dir)
for i in range(rows):
    for j in range(cols):
        food_dir = food_dirs[(i*cols + j)%101]
        all_files = os.listdir(os.path.join(train_dir, food_dir))
        img = plt.imread(os.path.join(train_dir, food_dir, all_files[3]))
        ax[i][j].imshow(img)
        ax[i][j].text(0, -20, food_dir, size=10, rotation=0, ha="left", va="top", 
                bbox=dict(boxstyle="round", ec=(0, .6, .1), fc=(0, .7, .2)))
plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    color_mode = "rgb",
    batch_size= 128,
    class_mode='categorical',
    shuffle = True)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), 
    color_mode = "rgb",
)

In [ ]:
#base_model = keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
#base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
#base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model = keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
#base_model = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(.25)(x)

predictions = Dense(101, activation='softmax')(x)

model = Model(base_model.input, predictions)

model.summary()

In [ ]:
model.compile(optimizer='Adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
      train_generator,
      epochs=70
)

In [ ]:
# Create {ID, filename} dictionary {'1269': '1269.jpg', '3863': '3863.jpg',.....}
test_dict = {}
for root, dirs, files in os.walk(root_dir + '/test'):
    for filename in files:
        test_id, file_ext = os.path.splitext(filename)
        test_dict[test_id] = filename

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load_img(filename, target_w=150, target_h=150):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (target_w, target_h, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image


In [ ]:
# Read images in order and make predictions
results = []
for i in range(len(test_dict)):
    img = load_img(root_dir + '/test/' + test_dict[str(i)], 150, 150)
    ret = model.predict(img)
    results.append(np.argmax(ret))

In [ ]:
# Print results in CSV format and upload to Kaggle
with open('pred_results.csv', 'w') as f:
    f.write('Id,Category\n')
    for i in range(len(results)):
        f.write(str(i) + ',' + str(results[i]) + '\n')

In [ ]:
# Download your results!
from IPython.display import FileLink
FileLink('pred_results.csv')

In [ ]:
import pandas as pd

pred_results_ = pd.DataFrame(columns=['Id','Category'])
for i,item in enumerate(results):
    pred_results_.loc[len(pred_results_)] = [i,item]
pred_results_.to_csv('pred_results_.csv')